In [34]:
import pandas as pd
import numpy as np

In [35]:
seeds = [0, 25, 50, 75, 100]
models = ['A2C', 'PPO', 'DDPG', 'SAC']

## portfolio values

In [36]:
for model in models:    
    data = pd.DataFrame()
    
    # Load all seed data (portfolio values are already cumulative from evaluation mechanism)
    for seed in seeds:
        portfolio_df = pd.read_csv(f'tables/{model.lower()}_multi_seed/{model}_seed_{seed}_portfolio.csv')
        portfolio_df['date'] = pd.to_datetime(portfolio_df['date'])
        
        # Extract iteration, train dates, test dates from metadata columns
        data[f'portfolio_values_{seed}'] = portfolio_df.set_index('date')['portfolio_value']

    # Calculate mean and standard deviation across seeds
    data[f'{model.lower()}_mean'] = data.mean(axis=1, numeric_only=True)
    data[f'{model.lower()}_sd'] = data.std(axis=1, numeric_only=True)
    data.to_csv(f'tables/{model.lower()}_multi_seed/{model}_cumulative_period_portfolio_values.csv')
    
    print(f"✓ {model} cumulative portfolio values aggregated from {len(seeds)} seeds")

✓ A2C cumulative portfolio values aggregated from 5 seeds
✓ PPO cumulative portfolio values aggregated from 5 seeds
✓ DDPG cumulative portfolio values aggregated from 5 seeds
✓ SAC cumulative portfolio values aggregated from 5 seeds


## weights

## turnovers

## transaction costs